# Sentiment Analysis

## Get the closing prices of Tesla

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
%matplotlib inline

# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2009-06-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-01-15", tz="America/New_York").isoformat()

# Set the tickers
ticker = "SNE"

# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

# Get current closing prices
df_closing_prices = alpaca.get_barset(
    ticker,
    timeframe,
    start = start_date,
    end = end_date
).df

# Dissolve multiindex and fetch the closing prices 
df_closing_prices = df_closing_prices.droplevel(0, axis=1)[['close']]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head()

,close
2009-06-01,27.37
2009-06-02,28.00
2009-06-03,27.40
2009-06-04,27.75
2009-06-05,27.84


In [2]:
df_closing_prices['return'] = df_closing_prices['close'].pct_change()
df_closing_prices.dropna(inplace=True)
df_closing_prices.head()

,close,return
2009-06-02,28.00,0.023018
2009-06-03,27.40,-0.021429
2009-06-04,27.75,0.012774
2009-06-05,27.84,0.003243
2009-06-08,27.55,-0.010417


## Load the data of google research results

In [3]:
import pandas as pd
from pathlib import Path
import calendar
from sklearn.preprocessing import LabelEncoder

file_path = Path("sony.csv")
news_df = pd.read_csv(file_path, parse_dates=[['date','time']], infer_datetime_format=True, usecols=['date',"tweet",'time'])

# Delete the index label and sort in ascending order
news_df.set_index('date_time', inplace=True)
news_df.index.name = None
news_df.sort_index(axis=0, inplace=True)

# Display sample data
print(len(news_df))
display(news_df.head())

20303


,tweet
2009-06-01 14:45:29,Heading to E3 for the first time; I'll keep yo...
2009-06-02 12:37:07,Just stepped into the convention center and we...
2009-06-02 13:00:18,The PlayStation booth is a massive two-level m...
2009-06-02 13:54:34,Sitting in the Shriner's Auditorium anxiously ...
2009-06-02 13:56:33,It's full house that's just getting as we wait.


In [4]:
# Drop news published after 4pm since it does not affect the closing price of that day
news_df = news_df.between_time('00:00:00','16:00:00', include_end=False)

# Drop time in the index labels
news_df.index = news_df.index.date

# Display sample data
print(len(news_df))
display(news_df.head())

15186


,tweet
2009-06-01,Heading to E3 for the first time; I'll keep yo...
2009-06-02,Just stepped into the convention center and we...
2009-06-02,The PlayStation booth is a massive two-level m...
2009-06-02,Sitting in the Shriner's Auditorium anxiously ...
2009-06-02,It's full house that's just getting as we wait.


## Sentiment Analysis

In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kn_na\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Create the Facebook Libra sentiment scores DataFrame
tweet_sentiments = []

for title in news_df["tweet"]:
    try:
        sentiment = analyzer.polarity_scores(title) # get sentiment score
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tweet_sentiments.append({"compound": compound,
                                 "positive": pos,
                                 "negative": neg,
                                 "neutral": neu
                                })
        
    except AttributeError:
        pass
    
# Create DataFrame
sentiments_df = pd.DataFrame(tweet_sentiments, index=news_df.index)

# Reorder DataFrame columns
news_df = news_df.join(sentiments_df)

news_df.head()

,tweet,compound,positive,negative,neutral
2009-06-01,Heading to E3 for the first time; I'll keep yo...,0.0000,0.000,0.000,1.000
2009-06-02,Just stepped into the convention center and we...,0.3818,0.157,0.000,0.843
2009-06-02,Just stepped into the convention center and we...,0.4588,0.235,0.000,0.765
2009-06-02,Just stepped into the convention center and we...,-0.2263,0.000,0.174,0.826
2009-06-02,Just stepped into the convention center and we...,0.0000,0.000,0.000,1.000


In [7]:
# sentiment_score_df = pd.DataFrame(index=news_df.index)
# sentiment_score_df.head()

In [8]:
# calculate the average of compound scores for a day with more than 1 article
import numpy as np
# sentiment_score_df = news_df.groupby(level=0)[['compound']].agg([('avg_sentiment',np.mean), ('article_counts',count)])
sentiment_score_df = news_df.groupby(level=0)[['compound']].count()
sentiment_score_df["avg_sentiments"] = news_df.groupby(level=0)[['compound']].mean()
#sentiment_score_df.rename(columns= {'compound':"article_counts"}, inplace=True)
sentiment_score_df.drop(columns='compound', inplace=True)
sentiment_score_df.head(10)

,avg_sentiments
2009-06-01,0.000000
2009-06-02,0.143471
2009-06-23,0.539500
2009-07-20,0.000000
2009-07-29,0.386233
2009-07-30,0.000000
2009-08-03,0.225436
2009-08-04,0.229400
2009-08-05,0.157729
2009-08-07,-0.119067


In [9]:
joined_df = pd.concat([df_closing_prices, sentiment_score_df], axis=1, join="inner")
joined_df.head()

,close,return,avg_sentiments
2009-06-02,28.00,0.023018,0.143471
2009-06-23,25.81,0.019755,0.539500
2009-07-20,24.65,0.014403,0.000000
2009-07-29,24.80,0.012658,0.386233
2009-07-30,27.29,0.100403,0.000000


In [10]:
#joined_df["class"] = joined_df["return"].apply(lambda x: 1 if x >= 0.01 else (0 if -0.01<x<0.01 else -1))
joined_df["class"] = joined_df["return"].apply(lambda x: 1 if x >= 0 else 0)

joined_df.head()

,close,return,avg_sentiments,class
2009-06-02,28.00,0.023018,0.143471,1
2009-06-23,25.81,0.019755,0.539500,1
2009-07-20,24.65,0.014403,0.000000,1
2009-07-29,24.80,0.012658,0.386233,1
2009-07-30,27.29,0.100403,0.000000,1


In [12]:
X=joined_df[['avg_sentiments']].values.reshape(-1,1)
y=joined_df[['class']].values.ravel()
print(X.shape)
print(y.shape)

(2482, 1)
(2482,)


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)
#print(X_scaled[:1])
# pipe = Pipeline([('scaler', StandardScaler()), ('pca',decomposition.PCA(n_components=2))])
# iris_X_scaled = pipe.fit_transform(iris_X)

## Concatenate the DFs of closing prices and sentiments

In [14]:
# # concatenate
# tesla_df = pd.concat([df_closing_prices, sentiment_score_df], axis=1, join='inner')

# tesla_df.head()

## Prepare Training and Testing data

In [15]:
# tesla_df["class"] = tesla_df["return"].apply(lambda x: 1 if x >= 0.01 else (0 if -0.01<x<0.01 else -1))

# tesla_df.head(15)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
            train_test_split(X, y, 
                             test_size=0.2,
                             random_state=1,
                             stratify=y
                            )

print(X_train.shape)
print(y_train.shape)

(1985, 1)
(1985,)


## Choosing and fitting models

In [17]:
# Compare the five models and choose the best one
# Logistic regression
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=1)

# Support vector machine
from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=1)

# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=1)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=1)

# XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=1)

# Create a list of the five model instances
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [18]:
# Compare the performance of the five models
for algorithm in algorithms:
    algorithm.fit(X_train, y_train)
    score = algorithm.score(X_test, y_test)
    name = algorithm.__class__.__name__
    
    print(f'{name} score: {score:.4f}')

LogisticRegression score: 0.5272
SVC score: 0.5352
DecisionTreeClassifier score: 0.5372
RandomForestClassifier score: 0.5433
[14:20:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier score: 0.5392


C:\Users\kn_na\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


## Cross Validation

In [19]:
# Create a list of the five model instances
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=1)

from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', gamma="scale", C=1, random_state=1)

from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=1)

from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=1)

from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=1)

algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [20]:
# Compare the performance of the five models

# Create a balanced set of samples, create a StratifiedKFold instance
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score

for algorithm in algorithms:
    # Conduct cross validation for each one of the five models
    scores = cross_val_score(algorithm, X_train, y_train, cv=stratifiedkfold)
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'{name} average score: {score:.4f} / each score: {scores}')

LogisticRegression average score: 0.5229 / each score: [0.51963746 0.52416918 0.52496218]
SVC average score: 0.5194 / each score: [0.51057402 0.51661631 0.53101362]
DecisionTreeClassifier average score: 0.5123 / each score: [0.51208459 0.52265861 0.50226929]
RandomForestClassifier average score: 0.5174 / each score: [0.51963746 0.52416918 0.50832073]
[14:20:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:20:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\kn_na\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:20:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier average score: 0.5214 / each score: [0.53776435 0.51208459 0.51437216]


## Grid Search

In [21]:
# Optimize the parameters
params = {'C':[1, 10, 100, 1000, 10000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}
algorithm = SVC(random_state=1)

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorithm, params, cv=stratifiedkfold)
gs.fit(X_train, y_train)

# Based on the best parameters, predict y values from test data
best = gs.best_estimator_
best_pred = best.predict(X_test)
print(best)

SVC(C=100, gamma=1, random_state=1)


In [22]:
# Accuracy
score = best.score(X_test, y_test)
print(f'score: {score:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix
print('confusion matrix')
print(confusion_matrix(y_test, best_pred))

score: 0.5292
confusion matrix
[[ 30 210]
 [ 24 233]]
